# Improving Pygments Code Block Display

Using Pygments, CSS and Ruff to improve how code blocks are displayed on my daily notebook blog.

In [163]:
from execnb.nbio import read_nb
from fasthtml.common import *
from fasthtml.jupyter import JupyUvi
from fastcore.utils import L
import pygments
from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import HtmlFormatter

## Notebook Root to L of Paths

In [162]:
root = Path() if IN_NOTEBOOK else Path("nbs/")
nb_paths = L(root.glob("*.ipynb")).sorted(reverse=True)
nb_paths

(#50) [Path('2025-01-26-Improving-Pygments-Code-Block-Display.ipynb'),Path('2025-01-25-This-Site-Is-Now-Powered-by-This-Notebook-Part-2.ipynb'),Path('2025-01-24-Creating-In-Notebook-Images-for-Social-Media-With-PIL-Pillow.ipynb'),Path('2025-01-23-Troubleshooting-MonsterUI-on-This-Site.ipynb'),Path('2025-01-23-This-Site-Is-Now-Powered-by-This-Notebook.ipynb'),Path('2025-01-22-MonsterUI-Buttons-and-Links.ipynb'),Path('2025-01-22-Customizing-FastHTML-Headers-From-Notebook-Contents.ipynb'),Path('2025-01-21-SVG-Animation-in-FastHTML.ipynb'),Path('2025-01-20-Dark-and-Light-Mode-in-FastHTML.ipynb'),Path('2025-01-19-Genanki-and-fastcore.ipynb'),Path('2025-01-18-Alarm-Sounds-App.ipynb'),Path('2025-01-17-Alarm-Clock-Sounds.ipynb'),Path('2025-01-16-Cosine-Similarity-Breakdown-in-LaTeX.ipynb'),Path('2025-01-14-Constructing-SQLite-Tables-for-Notebooks-and-Search.ipynb'),Path('2025-01-13-SQLite-FTS5-Tokenizers-unicode61-and-ascii.ipynb'),Path('2025-01-12-A-Better-Notebook-Index-Page.ipynb'),Path('20

## Get Sample Cells to Work With

In [103]:
nb = read_nb(nb_paths[10])

In [104]:
def is_code(c): return c.cell_type=='code'
c = L(nb.cells).filter(is_code)[3]
c

```json
{ 'cell_type': 'code',
  'execution_count': 16,
  'id': '00540ad9',
  'idx_': 10,
  'metadata': {},
  'outputs': [ { 'data': { 'text/plain': [ "(#3) [['Magandang hapon', 'Good "
                                           "afternoon'],['Magandang gabi', "
                                           "'Good evening'],['Paalam', "
                                           "'Goodbye']]"]},
                 'execution_count': 16,
                 'metadata': {},
                 'output_type': 'execute_result'}],
  'source': "notes = L(['Magandang hapon', 'Good afternoon'],\n"
            "    ['Magandang gabi', 'Good evening'],\n"
            "    ['Paalam', 'Goodbye'])\n"
            'notes'}
```

## Style the Cell

In [105]:
def StyledCode(c, style='monokai'):
    "A notebook cell styled as code, with style name as its css class for scope limiting"
    fm = HtmlFormatter(style=style, cssclass=style)
    h = highlight(c, PythonLexer(), fm)
    sd = fm.get_style_defs(f".{style}")
    return Div(Style(sd), NotStr(h), id=style)

In [106]:
show(StyledCode(c.source))

## Pad the Cell

In [138]:
def StyledCode(c, style='monokai'):
    "A notebook cell styled as code, with style name as its css class for scope limiting"
    fm = HtmlFormatter(style=style, cssclass=f"my-{style}", prestyles="padding:10px 0;")
    h = highlight(c, PythonLexer(), fm)
    sd = fm.get_style_defs(f".my-{style}")
    return Style(sd), NotStr(h)

Pygments lets you add inline styles to `<pre>`. That's nice.

In [139]:
st, sc = StyledCode(c.source)

In [140]:
Div(sc)

```html
<div><div class="my-monokai"><pre style="padding:10px 0;"><span></span><span class="n">notes</span> <span class="o">=</span> <span class="n">L</span><span class="p">([</span><span class="s1">&#39;Magandang hapon&#39;</span><span class="p">,</span> <span class="s1">&#39;Good afternoon&#39;</span><span class="p">],</span>
    <span class="p">[</span><span class="s1">&#39;Magandang gabi&#39;</span><span class="p">,</span> <span class="s1">&#39;Good evening&#39;</span><span class="p">],</span>
    <span class="p">[</span><span class="s1">&#39;Paalam&#39;</span><span class="p">,</span> <span class="s1">&#39;Goodbye&#39;</span><span class="p">])</span>
<span class="n">notes</span>
</pre></div>
</div>

```

In [141]:
show(StyledCode(c.source))

Success: the code is padded on top and bottom with 10px now.

I'd rather not pad the left and right of it any more than the container already does, because maximizing screen width is important to me. My thought is that having the Pygments style background color be close to the `<body>` background color in light and dark modes will suffice visually, but I have yet to test that.

## Limiting Line Length on Mobile Devices

On my phone I counted that 52 characters is the maximum I can read on a line of code, before having to scroll. Let's see if Ruff's textwrap breaks lines in a way that keeps Python code valid.

In [153]:
def wrap_text(text, width=52):
    wrapper = textwrap.TextWrapper(width=width)
    wrapped_text = wrapper.fill(text)
    return wrapped_text

In [154]:
wrapped_text = wrap_text(c.source)
print(wrapped_text)

notes = L(['Magandang hapon', 'Good afternoon'],
['Magandang gabi', 'Good evening'],     ['Paalam',
'Goodbye']) notes


In [160]:
wrapped_text = wrap_text(c.source, width=43)
print(wrapped_text)

notes = L(['Magandang hapon', 'Good
afternoon'],     ['Magandang gabi', 'Good
evening'],     ['Paalam', 'Goodbye']) notes


## Next Steps

If I continue this approach, the next steps will be:

* Find another way to wrap lines while ensuring they're still valid Python after wrapping
* Choose line length to return based on User-Agent header
* Put it all together